In [1]:
import pandas as pd
import json
ans = {}
dataset_list = ['banking', 'stackoverflow', 'clinc', 'hwu', 'mcid', 'news', 'ele']

pd_ans = {'dataset': [], 'ratio':[], 'split':[], 'value':[]}
for dataset_name in dataset_list:
    ans[dataset_name] = {}
    for split in ['train', 'dev', 'test']:
        for rate in [0.25, 0.5, 0.75]:
            ans[dataset_name][rate] = {}
            label_ratio = pd.read_csv(f'{dataset_name}/label/label_{rate}.list', header=None)[0]
            ratio = 0.1
            df = pd.read_csv(f"{dataset_name}/{split}.tsv", sep='\t')
            sub_df = df[df['label'].isin(label_ratio)]
            if split in ['train', 'dev']:
                labeled_data = pd.read_csv(f'{dataset_name}/labeled_data/{split}_{ratio}.tsv', sep='\t')['text'].tolist()
                sub_df = sub_df[sub_df['text'].isin(labeled_data)]
            ans[dataset_name][rate][split] = len(sub_df)
            ans[dataset_name][split] = len(df)
            if split in ['train', 'dev']:
                pd_ans['dataset'].append(dataset_name)
                pd_ans['split'].append(f"{split}")
                pd_ans['ratio'].append(rate)
                pd_ans['value'].append(len(sub_df))
            
            pd_ans['dataset'].append(dataset_name)
            pd_ans['split'].append('\#label')
            pd_ans['ratio'].append(rate)
            pd_ans['value'].append(len(label_ratio))

        pd_ans['dataset'].append(dataset_name)
        pd_ans['split'].append(f"{split}")
        pd_ans['ratio'].append(1)
        pd_ans['value'].append(len(df))
        
    num_labels = len(pd.read_csv(f'{dataset_name}/label/label.list', header=None)[0])
    pd_ans['dataset'].append(dataset_name)
    pd_ans['split'].append('\#label')
    pd_ans['ratio'].append(1)
    pd_ans['value'].append(num_labels)
    ans[dataset_name]['num_labels'] = num_labels
json.dump(ans, open('data_statics.json', 'w'), indent=4)
ans

<>:28: SyntaxWarning: invalid escape sequence '\#'
<>:39: SyntaxWarning: invalid escape sequence '\#'
<>:28: SyntaxWarning: invalid escape sequence '\#'
<>:39: SyntaxWarning: invalid escape sequence '\#'
/tmp/ipykernel_13857/154866126.py:28: SyntaxWarning: invalid escape sequence '\#'
  pd_ans['split'].append('\#label')
/tmp/ipykernel_13857/154866126.py:39: SyntaxWarning: invalid escape sequence '\#'
  pd_ans['split'].append('\#label')


/tmp/ipykernel_13857/154866126.py:28: SyntaxWarning: invalid escape sequence '\#'
  pd_ans['split'].append('\#label')
/tmp/ipykernel_13857/154866126.py:39: SyntaxWarning: invalid escape sequence '\#'
  pd_ans['split'].append('\#label')


FileNotFoundError: [Errno 2] No such file or directory: 'banking/label/label_0.25.list'

In [ ]:
pd_ans = pd.DataFrame(pd_ans).sort_values(['dataset', 'ratio', 'split'], ascending=[True, True, False]).drop_duplicates()
pivot_ans = pd_ans.pivot(index='dataset', columns=['ratio', 'split'], values=['value'])
pivot_ans = pivot_ans.applymap(lambda x: "{:,}".format(x))
pivot_ans.to_excel('data_statics.xlsx')
pivot_ans

/tmp/ipykernel_2238926/526809062.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_ans = pivot_ans.applymap(lambda x: "{:,}".format(x))


value                                                     \
ratio          0.25              0.50                0.75                
split         train dev \#label train  dev \#label  train  dev \#label   
dataset                                                                  
banking         193  23      19   430   50      38    682   79      58   
clinc           456  76      38   900  150      75  1,344  224     112   
hwu             183  25      16   365   52      32    595   83      48   
mcid             29   4       4    59    8       8     92   12      12   
news            296  50       5   555   93      10    841  142      15   
stackoverflow   300  50       5   600  100      10    900  150      15   

                                             
ratio            1.00                        
split           train   test    dev \#label  
dataset                                      
banking         9,000  3,073    999      77  
clinc          17,995  2,250  2,250     150  
hwu             7,712  1,032    933      64  
mcid            1,198    331    170      16  
news           11,287  5,653  1,879      20  
stackoverflow  11,996  5,991  1,998      20

In [ ]:
data_statics = pivot_ans.to_latex().replace(' ', '').replace('0000', '')
replace_map = {
    "Classes": '|Classes|',
    "train-labeled": '|Train|',
    "train": '|Train|',
    "eval": '|Validation|',
    "test": '|Test|',
    "banking": 'BANKING',
    "clinc": 'CLINC',
    "stackoverflow": 'STACK',
    "thucnews": 'CNEWS',
    "atis": 'ATIS',
    "snips": 'SNIPS',
    "ele": 'REVIEWS',
    "news": 'NEWSG',
    "0000": "",
    ".00": "",
    "lrrrrrrr": "l|p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}",
    "toprule\n": "toprule\nDataset"
}
for i, v in replace_map.items():
    data_statics = data_statics.replace(i, v)
print(data_statics.replace(' ', ''))

\begin{tabular}{llllllllllllll}
\toprule
Dataset&\multicolumn{13}{r}{value}\\
ratio&\multicolumn{3}{r}{0.25}&\multicolumn{3}{r}{0.50}&\multicolumn{3}{r}{0.75}&\multicolumn{4}{r}{1}\\
split&|Train|&dev&\#label&|Train|&dev&\#label&|Train|&dev&\#label&|Train|&|Test|&dev&\#label\\
dataset&&&&&&&&&&&&&\\
\midrule
BANKING&193&23&19&430&50&38&682&79&58&9,000&3,073&999&77\\
CLINC&456&76&38&900&150&75&1,344&224&112&17,995&2,250&2,250&150\\
hwu&183&25&16&365&52&32&595&83&48&7,712&1,032&933&64\\
mcid&29&4&4&59&8&8&92&12&12&1,198&331&170&16\\
NEWSG&296&50&5&555&93&10&841&142&15&11,287&5,653&1,879&20\\
STACK&300&50&5&600&100&10&900&150&15&11,996&5,991&1,998&20\\
\bottomrule
\end{tabular}



In [ ]:
import pandas as pd
ans = {}
pd_ans = {'dataset': [], 'ratio':[], 'split':[], 'value':[]}
for dataset_name in dataset_list:
    ans[dataset_name] = {}
    ans[dataset_name]['Classes'] = len(pd.read_csv(f"{dataset_name}/label/label.list", header=None))
    all_df = []
    for split in ['train', 'dev','test']:
        df = pd.read_csv(f"{dataset_name}/{split}.tsv", sep='\t')
        all_df.append(df)
        ans[dataset_name][f"{split}"] = int(len(df))
    all_df = pd.concat(all_df)
    all_df['len'] = all_df['text'].apply(lambda x: len(str(x).split(' ')))
    ans[dataset_name]['Length (Avg.)'] = all_df['len'].mean().round(2)

ans = pd.DataFrame(ans)
ans = ans.T
for col in ans.columns:
    if 'Length' not in col:
        ans[col] = ans[col].apply(lambda x: int(x))
ans

,Classes,train,dev,test,Length (Avg.)
banking,77,9000,999,3073,11.77
stackoverflow,20,11996,1998,5991,8.34
clinc,150,17995,2250,2250,8.31
hwu,64,7712,933,1032,6.60
mcid,16,1198,170,331,6.99
news,20,11287,1879,5653,306.82


In [ ]:
replace_map = {
    "Classes": '|Classes|',
    "train": '|Train|',
    "eval": '|Validation|',
    "test": '|Test|',
    "banking": 'BANKING',
    "clinc": 'CLINC',
    "stackoverflow": 'STACK',
    "thucnews": 'CNEWS',
    "atis": 'ATIS',
    "snips": 'SNIPS',
    "ele": 'REVIEWS',
    "news": 'NEWSG',
    "0000": "",
    ".00": "",
    "lrrrrrrr": "l|p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}",
    "toprule\n": "toprule\nDataset"
}
ans_latex = ans.to_latex()
for i, v in replace_map.items():
    ans_latex = ans_latex.replace(i, v)
print(ans_latex.replace(' ', ''))

\begin{tabular}{lrrrrr}
\toprule
Dataset&|Classes|&|Train|&dev&|Test|&Length(Avg.)\\
\midrule
BANKING&77&9000&999&3073&11.77\\
STACK&20&11996&1998&5991&8.34\\
CLINC&150&17995&2250&2250&8.31\\
hwu&64&7712&933&1032&6.60\\
mcid&16&1198&170&331&6.99\\
NEWSG&20&11287&1879&5653&306.82\\
\bottomrule
\end{tabular}

